# Anatomical (histology) dataset Colab Demo

- Demo core visulization in the paper (YT.Wei et al., 2026. Nature Communication)
- Retrograde tracing analysis (4 animals)
- Only take the output .csv after CCF registration pipeline
- First-time setup may take additional time for allen mouse model downloading

![Workflow](https://github.com/ytsimon2004/rscvp/blob/rev/docs/source/_static/histology_data_analysis_workflow.png?raw=true)

## Setup virtual environment

In [ ]:
# Install uv (fast Python package manager)
!curl -LsSf https://astral.sh/uv/install.sh | sh
!export PATH="$HOME/.local/bin:$PATH"

# Install aria2 for faster downloading
!apt-get update -qq
!apt-get install aria2 -qq

# Clone your repo and checkout to the release tag
!git clone https://github.com/ytsimon2004/rscvp.git
%cd rscvp
!git checkout wei2026-v1

# Install using uv
!uv pip install .[all]

In [ ]:
import logging

from rscvp.atlas.main_roi_atlas import RoiAtlasOptions
from rscvp.atlas.main_roi_quant_batch import RoiQuantBatchOptions
from rscvp.atlas.main_roi_query_batch import RoiQueryBatchOptions
from rscvp.atlas.main_roi_top_view import RoiTopViewOptions
from rscvp.util.util_demo import run_demo

In [ ]:
# silence non-important plotting warning
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

## Demo plots

### ROIs in slice view
- See registration results in all brain slices before 3D reconstruction
- Used in `Fig.6E`

In [ ]:
class ExampleRoiAtlas(RoiAtlasOptions):
    animal = 'YW043'
    cut_plane = 'coronal'
    debug_mode = True

    index = 0  # brain slice index from 0 to 68 in this example animal

    def foreach_slice(self, *args, **kwargs):
        from itertools import islice
        all_slices = super().foreach_slice(*args, **kwargs)
        return [next(islice(all_slices, self.index, self.index + 1))]


run_demo(ExampleRoiAtlas, clean_cached=False)

### ROIs in dorsal cortex view
- See cortical distribution of different projection pattern
- Used in `Fig.6G`

In [ ]:
class ExampleRoiTopView(RoiTopViewOptions):
    animal = ('YW043', 'YW051', 'YW063', 'YW064')
    area_family = 'ISOCORTEX'
    legend_number_limit = 80
    ibl_res = 50

    def plot_ibl_view(self, ax, ibl):
        # if with sufficient RAM (colab free user may not), uncomment return below
        # return super().plot_ibl_view(ax, ibl)
        ...


run_demo(ExampleRoiTopView, clean_cached=False)

### Dots plot for all brain regions
- See summary of different projections from all regions
- Used in `Fig.6H`

In [ ]:
class ExampleDotPlot(RoiQuantBatchOptions):
    animal = ('YW043', 'YW051', 'YW063', 'YW064')
    dispatch_plot = 'heatmap'
    top_area = 60
    merge_level = 2
    roi_norm = 'channel'


run_demo(ExampleDotPlot, clean_cached=False)

### Subregion horizontal barplot
- Percentage contribution of each subregion within an area to specific RSC inputs
- Example shows projection to parahippocampal subregions
- Used in `Fig.7E`, `Fig.8A-8B`

In [ ]:
class ExampleSubregion(RoiQueryBatchOptions):
    animal = ('YW043', 'YW051', 'YW063', 'YW064')
    area = 'RHP'
    debug_mode = True


run_demo(ExampleSubregion, clean_cached=False)

### 3D reconstruction in selected brain regions
- Visualization of spatial distribution, example shows rois in visual areas in a animal
- Used in `Fig.7`, `Fig.8`
- Seealso [customized brainrender wrapper doc](https://neuralib.readthedocs.io/en/latest/atlas/brainrender.html#flexible-reconstruction-processed-csv)
- Brainrender can’t open its interactive 3-D VTK window because Colab runs in a headless, sandboxed environment that blocks GUI and WebGL execution inside iframes? Workaround: Download the output.html to visualization, otherwise, use local env.

In [ ]:
!nl_brainrender roi \
--classifier - file
~ /.cache / rscvp / rscvp_dataset / analysis / hist / YW043 / resize / processed / transformations / labelled_regions / parsed_data / parsed_roi.csv \
    - -only - source
aRSC, pRSC, overlap \
            - -roi - color
palegreen, magenta, orange \
                    - -roi - region
VISal, VISam, VISl, VISli, VISp, VISpl, VISpm, VISpor \
                                               - -region
VISal, VISam, VISl, VISli, VISp, VISpl, VISpm, VISpor \
                                               - -no - root \
                                               - -source - order
aRSC, pRSC, overlap \
            - -output
example.html


### Bias index plot
- See the bias projection brain area the either anterior versus posterior RSC
- Used in `Fig.8D`

In [ ]:
class ExampleBiasIndex(RoiQuantBatchOptions):
    animal = ('YW043', 'YW051', 'YW063', 'YW064')
    dispatch_plot = 'bias_index'
    top_area = 60
    merge_level = 2
    roi_norm = 'channel'


run_demo(ExampleBiasIndex, clean_cached=False)